<a href="https://colab.research.google.com/github/Scottzeng03/scott1040/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 3.9 MB/s eta 0:00:00


In [6]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    VideoMessage,
    LocationMessage,
    ImageMessage

)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
    response = model.generate_content(question)
    return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == "sticker":
          #回復貼圖
          reply = StickerMessage(
            package_id='446',
            sticker_id='1989'
         )
        elif action =='image':
          #回復圖片
          reply = ImageMessage(
            original_content_url='https://media.istockphoto.com/id/1205203628/photo/massive-traffic-at-rush-hour-in-taipei-downtown-taiwan.jpg?s=2048x2048&w=is&k=20&c=X79yV4COp7h-OcKvsdWyIfSAJv5aozW_nFgusmmdWAs=',
            preview_image_url='https://media.istockphoto.com/id/1205203628/photo/massive-traffic-at-rush-hour-in-taipei-downtown-taiwan.jpg?s=2048x2048&w=is&k=20&c=X79yV4COp7h-OcKvsdWyIfSAJv5aozW_nFgusmmdWAs='
         )
        elif action =='video':
          #回復影片
          reply = VideoMessage(
            original_content_url='https://media.istockphoto.com/id/1344914766/video/fly-over-the-city.mp4?s=mp4-640x640-is&k=20&c=vL3FqgoODLLopDgVT_pAj40w-9Ht7NLV65Ic_JvqMT8=',
            preview_image_url='https://media.istockphoto.com/id/1344914766/video/fly-over-the-city.mp4?s=mp4-640x640-is&k=20&c=vL3FqgoODLLopDgVT_pAj40w-9Ht7NLV65Ic_JvqMT8='
        )
        elif action =='location':
          #回復地點
          reply = LocationMessage(
            title='Taipei 101',
            address='110台北市信義區信義路五段7號',
            latitude=25.034112070859265,
            longitude=121.56451207398551
        )
        else:
         response = ask_gemini(action)
         reply = TextMessage(text=response)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                   reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://928f-35-236-254-125.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:12:09] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:12:10] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:12:15] "GET /callback HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:12:27] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:12:37] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:12:52] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:13:01] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:13:24] "POST /callback HTTP/1.1" 200 -
